# Project Template: Regression Experiments using sentence embeddings

ISE 201, section 33

28 Mar 2024  JM Agosta

_A template to follow for your final class project._

Your presentation from this notebook should be divided into sections. Each section starts with a markdown cell that mentions the purpose of the following cells. Pls divide your computation into manageably sized cells. You can check intermediate results by an expression in the last line of the cell -- no need for a print function to enclose them.

The goal of this project is to consolidate the work you've done over the past projects, to run a regression experiment using sentence embedding data. An experiment works by

- picking an evaluation measure (a "statistic"),
- running a baseline evaluation of the regression,
- coming up with a presumed improvement to the regression that could be a generation or selection of features going into the regression,
- making a comparative evaluation to see if the improvement is "statistically" significant.

You can refer to the previous class notebooks that can be found on Google Drive, and feel free to share ideas that others have contributed in the class, that can be found in the notebooks they've shared.  

Design your notebook, with sections and visualizations that are self-explanatory (label your graph axes!) so that you make the best use of the 8 minutes you'll have to present.

## Setup. Import libraries.  Check if any need to be loaded that are missing.

In [1]:
import pandas as pd
import numpy as np
import re, os
# For vector norms, and determinants.
import numpy.linalg as la
# Plotting
import matplotlib as plt
import seaborn as sns

# javascript plots
from bokeh.plotting import figure, show
from bokeh.models import BoxAnnotation, ColumnDataSource, VBar, Span, Legend
from bokeh.io import output_notebook
from bokeh.palettes import Category10
output_notebook()

import sklearn.linear_model as sl

# Distributions
from scipy.stats import norm

# Principal component analysis
from sklearn.decomposition import PCA, TruncatedSVD
# The random number generator can be used to select a random set of columns for visualization
from numpy.random import Generator, PCG64
rng = Generator(PCG64())

Loading BokehJS ...

In [2]:
# use statsmodels to compute OLS regression standard errors
# since scikit learn doesn't compute them.
# This update fails
# os.system('python -m pip install git+https://github.com/statsmodels/statsmodels')
import statsmodels.api as sm
sm.__version__

'0.15.0.dev270+gc22837f06'

### Set globals

In [3]:
# Global variables
EMBEDDING_DIMENSIONS = 384     # This model creates vectors of this length.
TEST_SET_SIZE = 500
BOOTSTRAP_RESAMPLES = 200
RELOAD_PACKAGE = False         # Reload the embedding package
USE_COLAB = False              # Load the data from your Google Drive

In [4]:
# If you need to rerun the sentence embeddings transformer, you'll have to reload the package since
# its not included in Colab.  Set RELOAD_PACKAGE to True to reload it.


# We'll use this dataset of 3000 labeled text samples.
# https://archive.ics.uci.edu/dataset/331/sentiment+labelled+sentences

# For converting the text into vectors, use this llm package
# See https://pypi.org/project/sentence-transformers/
try:
  from sentence_transformers import SentenceTransformer
except:
  if RELOAD_PACKAGE:
    os.system('python -m pip install sentence-transformers')
    from sentence_transformers import SentenceTransformer
  else:
      print('import SentenceTransformer failed')


## Load the data, and run the sentence tranformers on it, if needed

We've been using the data set as indexed by embeddings from ---a set of 3,000 items from 3 different sources, - "IMDB", Amazon, and Yelp reviews, on which the 'all-MiniLM-L6-v2' sentenceTransformer model generates 384 features.

If you need to download this dataset, go to:

    https://archive.ics.uci.edu/dataset/331/sentiment+labelled+sentences

If you want to use a larger set, here's an NLP data set from Kaggle  of 50,000 items. One can find them on the Kaggle site:

To download the dataset, go to:

https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

You would need to download the "IMDB Dataset.csv" file - 66.21MB


In [5]:
# Run this if you want to run on Colab and use the already featurized data

if USE_COLAB and not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive/')
  os.chdir('/content/drive/MyDrive/ISE201/notebooks/data')
else:
  os.chdir('./sentiment_labelled_sentences')
os.listdir(os.getcwd())

['.DS_Store',
 'amazon_cells_labelled.txt',
 'imdb_labelled.txt',
 'readme.txt',
 'sentence_embeddings.parquet',
 'yelp_labelled.txt']

In [6]:
# Load the embedding vectors dataset
if os.environ['HOME'] == '/home/jma':
    featurized_parquet_file = 'sentence_embeddings.parquet'
elif USE_COLAB:
  # Find the file on Googe Drive
    featurized_parquet_file = './3000_sentence_embeddings.parquet'

text_df = pd.read_parquet(featurized_parquet_file)

# Extract the outcome y vector
outcomes = text_df['sentiment'].values

# Extract the vector field and expand it to multiple rows.
n_samples, embedding_dim = text_df.shape
data_ar = np.empty((n_samples, EMBEDDING_DIMENSIONS), 'float')
for i in range(n_samples):
    x = text_df.values[i,2]
    data_ar[i] = x

# Check the data shapes: raw NLP data, and itsEMB sentence embeddings, outocomes
text_df.shape, data_ar.shape, outcomes.shape

((2748, 3), (2748, 384), (2748,))

In [28]:
# Split the data into a learning and holdout set
# Create a random sample, with replacement
sample_size = outcomes.shape[0]- TEST_SET_SIZE
learning_set = np.random.choice(len(data_ar),\
                                sample_size,\
                                replace=False)
learning_data = data_ar[learning_set,:]
learning_outcome = text_df['sentiment'].values[learning_set]

# Run predictions against a hold-out set
not_learning_set = list(set(text_df.index).difference(set(learning_set)))  # TODO check if this set is correct
holdout_data = data_ar[not_learning_set,:]
# holdout_data = sm.add_constant(holdout_data)  # TODO - Make this consistent 
holdout_actuals = text_df['sentiment'].values[not_learning_set]
learning_data.shape, learning_outcome.shape, holdout_data.shape, holdout_actuals.shape

((2248, 384), (2248,), (171, 384), (171,))

## Pick an evaluation metric & create visualization graphics

In [29]:
# An in-sample metric
def AIC(the_fit):
  'Use this to compute AIC on different models.'
  return 2* (the_fit.df_model+1) - 2*the_fit.llf

# A metric to compare out-of-sample error
def one_zero_error(predicted, actual, test_threshold = 0.5):
  'The predicted is a continuous value, so it needs to be thresholded. '
  test_df = pd.DataFrame(dict(prediction=pd.Series(predicted >= test_threshold), actual=actual))
  test_error = sum(test_df.prediction != actual)/len(actual)
  return test_error

In [30]:
# A plot that shows the difference between the normal approximation interval and an equivalent coverage empirical interval,

def expand_range(low, high, by=2.0):
    mid = (low + high)/2
    half_width = by * abs(high - low)/2
    return (mid - half_width, mid + half_width)

# Compute two different interval estimates
# The first assuming a normal distribution, using standard deviation
# The second computing the equivalent quantiles 0.025 and 0.975, from the empirical data. 
# Note: For extreme distributions, such as those on a closed or half interval, this may not be the best approximation
def interval_estimate(data_list):
    data_ar = np.array(data_list)
    mn = data_ar.mean()
    sd = data_ar.std()
    return {'mean': mn, 'sd': sd,
        'normal_interval': (mn-1.96*sd, mn+1.96*sd),
            'quantile_interval': (np.percentile(data_ar, 2.5), np.percentile(data_ar, 97.5))}

def plot_intervals(the_sample, bin_ct = 50, subtitle = ''):
    normal_approx = interval_estimate(the_sample)
    lower_limit = min(normal_approx['normal_interval'][0], normal_approx['quantile_interval'][0])
    upper_limit = max(normal_approx['normal_interval'][1], normal_approx['quantile_interval'][1])
    lower_plot_limit, upper_plot_limit = expand_range(lower_limit, upper_limit)
    print('Limits ', lower_plot_limit, upper_plot_limit)
    p = figure(width = 800, height = 300, x_range = (lower_plot_limit,  upper_plot_limit ), title='Normal versus Empirical Bootstrap Interval Estimates.\n' + subtitle)

    hist, bin_edges = np.histogram(the_sample, bins=bin_ct, density=True)
    hist_df = pd.DataFrame(dict(density=hist, rv= bin_edges[:-1]), columns=['rv', 'density'])
    # Shade the background corresponding to the normal interval
    box = BoxAnnotation(left=normal_approx['normal_interval'][0], right=normal_approx['normal_interval'][1], bottom=0, top=max(1, max(hist)), fill_alpha=0.2, fill_color='#D55E00')
    p.add_layout(box)
    bar_width = (bin_edges[-1] - bin_edges[0])/bin_ct
    hist_src = ColumnDataSource(hist_df)

    glyph = VBar(x='rv', top='density', bottom=0, width=bar_width, fill_color='limegreen')
    p.add_glyph(hist_src, glyph)
    # Overlay a selection of the histogram within the estimate interval
    interval_src = ColumnDataSource(hist_df[(hist_df.rv > normal_approx['quantile_interval'][0]) & (hist_df.rv < normal_approx['quantile_interval'][1])])
    glyph = VBar(x='rv', top='density', bottom=0, width=bar_width, fill_color='darkgreen')
    p.add_glyph(interval_src, glyph)
    # Overlay the normal distribution within the interval.
    x = np.linspace(normal_approx['normal_interval'][0], normal_approx['normal_interval'][1])
    p.line(x=x, y=norm.pdf(x, loc= normal_approx['mean'], scale = normal_approx['sd']))
    return p

## Run the baseline regression

In [31]:
# Run an OLS regression on the full data.
def regress(y, X):
  design_matrix = sm.add_constant(X)
  ols_model = sm.OLS(y, design_matrix)
  #print(ols_model.fit().summary())
  return ols_model.fit()

def print_eval_summary(the_fit):
  # Just show the evaluation table, without coefs (tables[1] give coefs)
  s = the_fit.summary()
  print (s.tables[0])
  return None

In [32]:
fit = regress(outcomes, data_ar)
print_eval_summary(fit)
'AIC: ', AIC(fit)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.662
Method:                 Least Squares   F-statistic:                     15.06
Date:                Mon, 22 Apr 2024   Prob (F-statistic):               0.00
Time:                        09:56:21   Log-Likelihood:                -299.62
No. Observations:                2748   AIC:                             1365.
Df Residuals:                    2365   BIC:                             3632.
Df Model:                         382                                         
Covariance Type:            nonrobust                                         


('AIC: ', 1365.2353739571927)

In [33]:
# Compute in-sample error
one_zero_error(fit.predict(), outcomes, test_threshold = 0.5)

0.06186317321688501

In [40]:
# Look at some of the beta coefficients
pd.DataFrame(fit.summary().tables[1])

,0,1,2,3,4,5,6
0,,coef,std err,t,P>|t|,[0.025,0.975]
1,const,0.5822,0.056,10.336,0.000,0.472,0.693
2,x1,-0.2610,0.235,-1.112,0.266,-0.721,0.199
3,x2,0.4099,0.237,1.727,0.084,-0.056,0.875
4,x3,-0.2591,0.268,-0.967,0.333,-0.784,0.266
...,...,...,...,...,...,...,...
381,x380,-0.0595,0.275,-0.216,0.829,-0.599,0.480
382,x381,0.5904,0.229,2.576,0.010,0.141,1.040
383,x382,0.8383,0.230,3.650,0.000,0.388,1.289
384,x383,0.1760,0.231,0.760,0.447,-0.278,0.630


## An example Interval estimate for out-of-sample 0-1 error via Bootstrap re-sampling

This demonstrates the 0-1 error rate interval estimate, by resampling the training predictor, then measuring accuracy on the holdout set. This may be overkill, as a way to estimate the variation in the holdout error. 

Note that the non-parametric interval does slightly worse than its normal approximation interval. 

In [35]:
# Create a large number of bootstrap replications
# Each column in "resamplings" is a set of indexes into the rows of the training set, resampled with replacement.

def bootstrap_replicates(the_sample, B=BOOTSTRAP_RESAMPLES):
    replicates = np.zeros((len(the_sample), B), dtype='int')
    for k in range(B):
        replicates[:,k] = np.random.choice(the_sample, len(the_sample), replace=True).astype('int')
    return replicates

resamplings = bootstrap_replicates(list(range(sample_size)))
resamplings.shape

(2248, 200)

In [36]:
# How much duplication is there in the resamplings? 
# Remove duplicates by looking at the size of the resamplings as sets
resampling_size = [len(set(resamplings[:,k])) for k in range(resamplings.shape[1])]
pd.DataFrame(resampling_size).describe()

,0
count,200.000000
mean,1421.115000
std,15.175416
min,1364.000000
25%,1411.750000
50%,1421.000000
75%,1431.000000
max,1466.000000


In [37]:
# What are the estimates on the learning data?
# fit = regress(learning_outcome, learning_data[:,:100])
xols_model = sm.OLS(learning_outcome, learning_data[:,:EMBEDDING_DIMENSIONS])
xfit = xols_model.fit()
print_eval_summary(xfit)
predict_with_data = np.copy(holdout_data[:,:EMBEDDING_DIMENSIONS])
hold_out_predict = xfit.predict(predict_with_data)
print(f'hold out error {one_zero_error(hold_out_predict, holdout_actuals):.4}')
baseline_AIC = AIC(xfit)
'AIC: ', baseline_AIC

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.853
Model:                            OLS   Adj. R-squared (uncentered):              0.823
Method:                 Least Squares   F-statistic:                              28.44
Date:                Mon, 22 Apr 2024   Prob (F-statistic):                        0.00
Time:                        09:56:23   Log-Likelihood:                         -255.36
No. Observations:                2248   AIC:                                      1275.
Df Residuals:                    1866   BIC:                                      3459.
Df Model:                         382                                                  
Covariance Type:            nonrobust                                                  
hold out error 0.1404


('AIC: ', 1276.7224581467462)

In [42]:
holdout_w_const.shape

(171, 385)

In [43]:
# Bootstrap the zero-one error on a hold-out set.

# Create a large number of bootstrap replications
resampled_errors = np.zeros(BOOTSTRAP_RESAMPLES)

# Retrain the model on the resampled learning set,
# and compute the hold-out set error for each resample
print('learning AIC: ')
for k in range(BOOTSTRAP_RESAMPLES):
  resampled_data  = learning_data[resamplings[:,k],:]
  retraining_fit = regress(learning_outcome[resamplings[:,k]], resampled_data) # [resamplings[:,k]], resampled_data)
  print(f'{AIC(retraining_fit):.5}', end=', ')
  if k % 10 == 0:
    print()

  # Predict on the hold-out set
  holdout_w_const = sm.add_constant(np.copy(holdout_data))
  hold_out_predict = retraining_fit.predict(holdout_w_const)
  resampled_errors[k] = one_zero_error(hold_out_predict, holdout_actuals)


learning AIC: 
777.16, 
693.49, 625.04, 711.36, 846.61, 738.81, 701.46, 698.18, 749.25, 755.2, 620.32, 
548.93, 691.55, 736.63, 690.52, 699.75, 659.17, 729.54, 667.24, 798.8, 767.93, 
743.16, 775.72, 615.21, 710.53, 741.22, 695.2, 839.41, 884.19, 677.5, 622.69, 
869.48, 830.9, 746.25, 803.88, 707.33, 584.77, 763.84, 750.1, 639.51, 710.58, 
640.86, 734.47, 685.49, 827.89, 746.31, 794.2, 821.2, 694.43, 710.82, 633.18, 
698.64, 625.89, 544.26, 691.14, 836.76, 733.55, 635.94, 675.87, 791.64, 734.51, 
640.28, 816.49, 677.78, 707.06, 779.88, 677.93, 823.15, 639.89, 789.02, 676.44, 
840.44, 699.82, 691.91, 806.98, 798.28, 694.6, 660.38, 857.06, 817.44, 751.11, 
871.2, 829.38, 783.82, 703.72, 835.33, 703.64, 827.76, 718.33, 734.81, 815.06, 
648.38, 633.75, 864.07, 690.05, 802.71, 638.42, 719.63, 688.62, 681.35, 592.17, 
603.24, 708.47, 635.81, 702.52, 661.1, 534.71, 803.28, 731.11, 610.62, 843.19, 
705.96, 844.29, 644.72, 782.79, 792.42, 622.65, 822.57, 680.15, 722.64, 637.56, 
715.25, 630.1, 

In [44]:
# What to the bootstrap errors look like?
pd.DataFrame(resampled_errors).describe()

,0
count,200.000000
mean,0.151579
std,0.018338
min,0.105263
25%,0.140351
50%,0.152047
75%,0.163743
max,0.192982


In [45]:
#Plot the error distribution, and compute the interval estimate limit.
p = plot_intervals(resampled_errors)
show(p)

Limits  0.07987562142127524 0.22328227331556683


##  Generate selected features, for an alternative model

A crude way to select features for linear regression is to sort the variables by the largest correlation with the dependent variable.  Let's see how the holdout error changes as variables are added to the linear regression. 

In [46]:
# Use a dot product to approximate the covariance

feature_cov = learning_data.T @ learning_outcome / la.norm(learning_data, axis=0)
pd.DataFrame(feature_cov).describe()

,0
count,384.000000
mean,0.079746
std,8.490415
min,-22.757116
25%,-6.780350
50%,0.056663
75%,6.590051
max,18.940927


In [47]:
# Ordering of the variables by absolute value of cov
feature_ordering =   np.argsort(feature_cov) # np.argsort(np.abs(feature_cov)) #
ordered_features = feature_cov[feature_ordering[::-1]]        # [::-1] means reverse the ordering
pd.DataFrame(ordered_features[:10])

,0
0,18.940927
1,17.786689
2,17.507969
3,17.034517
4,16.914401
5,16.908888
6,16.579352
7,16.398230
8,16.115351
9,15.813918


In [60]:
# # Compute AIC as a function of the number of variables, using no selection ordering
unsorted_AIC = np.zeros(380)
for var_count in range(8,380):
    fit = regress(learning_outcome, learning_data[:,:var_count]) 
    unsorted_AIC[var_count] = AIC(fit)

In [57]:
# Compute AIC as a function of the number of variables
sorted_AIC = np.zeros([380, 3])
# Hmm - using the negative valued cov does slightly better. 
sorted_learning_data = learning_data[:,feature_ordering] # learning_data[:,feature_ordering[::-1]]
for var_count in range(8, 380):
    fit = regress(learning_outcome, sorted_learning_data[:,:var_count]) 
    sorted_AIC[var_count, 0] = var_count
    sorted_AIC[var_count, 1] = AIC(fit)
    sorted_holdout_data = holdout_data[:,feature_ordering]
    hold_out_predict = fit.predict(sm.add_constant(sorted_holdout_data[:,:var_count]))
    sorted_AIC[var_count, 2] = one_zero_error(hold_out_predict, holdout_actuals)
pd.DataFrame(sorted_AIC).describe()

,0,1,2
count,380.000000,380.000000,380.000000
mean,189.426316,1424.479434,0.151462
std,109.966473,376.093109,0.038231
min,0.000000,0.000000,0.000000
25%,94.750000,1222.951303,0.134503
50%,189.500000,1268.268478,0.140351
75%,284.250000,1673.534886,0.169591
max,379.000000,2776.655685,0.315789


In [58]:
f = figure(width = 700, height = 300,  x_range = (8,  EMBEDDING_DIMENSIONS ), y_range=(0.8*unsorted_AIC, max(sorted_AIC[:,1])), title='AIC for cov-selected variable sets') # ,
f.line(x=list(range(380)), y=unsorted_AIC, color = 'darkred')
f.line(x=sorted_AIC[:,0], y=sorted_AIC[:,1], color = 'red')
f.line(x=sorted_AIC[:,0], y=10000 * sorted_AIC[:,2], color = 'navy')
hline = Span(location= baseline_AIC, dimension='width', line_width= 0.6) # , fill_alpha=0.4)
f.renderers.extend([hline])
show(f)

ValueError: failed to validate FigureOptions(...).y_range: expected an element of either Instance(Range), Either(Tuple(Float, Float), Tuple(Datetime, Datetime), Tuple(TimeDelta, TimeDelta)), Seq(String), Object(Series) or Object(GroupBy), got (array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
       2289.53700392, 2242.6769956 , 2243.04934518, 2233.71414162,
       2221.02587075, 2222.49871302, 2219.70001953, 2095.03171881,
       2092.51698339, 2044.98818698, 2046.43601049, 2024.03935368,
       1987.18116368, 1969.28865966, 1967.50631605, 1929.93275349,
       1931.12472728, 1925.90519234, 1925.20100571, 1907.18048955,
       1906.93841893, 1818.41364693, 1819.70280931, 1819.37467184,
       1814.6682452 , 1815.6490535 , 1790.13037968, 1768.84352747,
       1761.6722119 , 1763.23853949, 1722.52472187, 1714.86308359,
       1710.63086567, 1707.71894438, 1700.44042998, 1677.43042614,
       1670.6178095

## Compare baseline and alternative models

In [ ]:
#TBD